# MULTILAYER PERCEPTRON MODEL

#### 1) Implementation of Perceptron Algorithm

In [ ]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# class for Multilayer Perceptron
class MLPAlgorithm(object):

    # hyperparameters definition
    def __init__(self, eta, threshold, max_epochs):
        self.eta = eta
        self.threshold = threshold
        self.max_epochs = max_epochs

    # function to define MLP architecture
    def build_architecture(self, input_length, hidden_length, output_length):
        # hyperparameters for MLP architecture
        self.input_length = input_length
        self.hidden_length = hidden_length
        self.output_length = output_length

        # random initialization of weights and biases
        # hidden layer [b x (a+1)]
        self.Wh = np.random.rand(self.hidden_length, self.input_length)
        self.bh = np.random.rand(self.hidden_length)
        # output layer [c x (b+1)]
        self.Wo = np.random.rand(self.output_length, self.hidden_length)
        self.bo = np.random.rand(self.output_length)

        return self

    # linear combination
    def input_net(self, x, w, b):
        net = np.dot(w, x) + b
        return net

    # sigmoid activation function
    def f(self, net):
        return 1/(1 + math.exp(-net))

    # derivate of sigmoid activation function
    def df(self, fnet):
        return fnet * (1 - fnet)

    # forward step
    def forward(self, x):
        # hidden layer
        self.net_h = self.input_net(x, self.Wh, self.bh)
        self.fnet_h = np.array([self.f(net) for net in self.net_h])
        self.dfnet_h = np.array([self.df(fnet) for fnet in self.fnet_h])

        # output layer
        self.net_o = self.input_net(self.fnet_h, self.Wo, self.bo)
        self.fnet_o = np.array([self.f(net) for net in self.net_o])
        self.dfnet_o = np.array([self.df(fnet) for fnet in self.fnet_o])

    # function to make prediction given a point
    def predict(self, xi):
        self.forward(xi)
        logits = self.fnet_o

        if self.output_length == 1:
            pred = np.where(self.fnet_o >= 0.5, 1, 0)[0]
        else:
            pred = np.argmax(self.fnet_o)[0]

        return logits, pred

    # iterative training step
    def fit(self, x_train, y_train):
        n = x_train.shape[0]
        E = 2 * self.threshold
        count = 0
        cost = list()

        # training in each epoch
        while(E >= self.threshold and count <= self.max_epochs + 1):
            E = 0

            # stochastic gradient descendent algorithm (SGD) -> for each sample
            for i in range(n):
                xi = x_train[i, :]

                if self.output_length > 1:  # for multi-classification problems
                    yi = y_train[i, :]
                else:                        # for binary-classification problems
                    yi = y_train[i]

                self.forward(xi)
                y_pred = self.fnet_o

                # calculate error
                error = (yi - y_pred)
                E = E + sum(error**2)

                #Output Layer
                #------------#

                # initialize gradient weights Wo for output layer
                dE_dWo = np.zeros(
                        self.output_length * self.hidden_length).reshape(
                                self.output_length, self.hidden_length)

                # initialize gradient bias bo for output layer
                dE_dbo = np.zeros(self.output_length)

                # calculate delta for output layer
                delta_o = -error * self.dfnet_o

                # iterate for each neuron in output layer
                for j in range(self.output_length):
                    # iterate in each sinapsis related with j-th output neuron
                    for i in range(self.hidden_length):
                        # dE_dWo
                        dE_dWo[j, i] = delta_o[j] * self.fnet_h[i]
                        self.Wo[j, i] = self.Wo[j, i] - self.eta * dE_dWo[j, i]
                    # dE_dbo
                    dE_dbo[j] = delta_o[j] * 1
                    self.bo[j] = self.bo[j] - self.eta * dE_dbo[j]

                #Hidden Layer
                #------------#

                # initialize gradient weights Wh for hidden layers
                dE_dWh = np.zeros(
                        self.hidden_length * self.input_length).reshape(
                                self.hidden_length, self.input_length)

                # initialize gradient biases bh for hidden layers
                dE_dbh = np.zeros(self.hidden_length)

                # calculate delta for hidden layer
                delta_h = self.dfnet_h * np.dot(delta_o, self.Wo)

                # iterate for each neuron of hidden layers
                for i in range(self.hidden_length):
                    # iterate in each sinapsis related with i-th hidden neuron
                    for k in range(self.input_length):
                        #dE_dWh
                        dE_dWh[i, k] = delta_h[i] * xi[k]
                        self.Wh[i, k] = self.Wh[i, k] - self.eta * dE_dWh[i, k]
                    #dE_dbh
                    dE_dbh[i] = delta_h[i] * 1
                    self.bh[i] = self.bh[i] - self.eta * dE_dbh[i]

            # count number of epochs
            count = count + 1

            # calculate mean square error (MSE)
            E = round(1/2 * (E/n), 5)
            cost.append(E)

            # report results each 100 epochs
            if(count%100 == 0):
                print('Epoch ', count, ': loss = ', E)

        # store results in MLP class
        self.epochs = count
        self.loss_ = E
        self.cost_ = cost

        return self

    # function to make iterative process of test
    def test(self, x_test, y_test):
        n = x_test.shape[0]
        self.accuracy = 0
        y_prob = list()
        y_pred = list()

        for i in range(n):
            xi = x_test[i, :]

            if self.output_length > 1:  # for multi-classification problems
                yi = y_test[i, :]
            else:                       # for binary classification problems
                yi = y_test[i]

            # given inputs, predict class and probabilities
            logits, pred = self.predict(xi)
            y_prob.append(logits)
            y_pred.append(pred)

            # verify correct classifications
            if np.array_equal(y_pred[i], yi):
                self.accuracy += 1

        # calculate accuracy
        self.accuracy = 100 * round(self.accuracy/n, 5)

        return y_pred


#### 2) Preprocessing

In [ ]:
# load dataset
dataset = pd.read_csv("dataset/moonDataset.csv", header = None)
dataset.columns = ['X1','X2','X3','Outcome']
dataset.head()

,X1,X2,X3,Outcome
0,-0.926767,-0.111073,0.086017,0
1,-0.917583,0.706006,0.058041,0
2,0.437984,0.899093,0.072543,0
3,0.089694,0.291446,0.070444,1
4,0.110672,-0.070806,-0.090376,1


In [ ]:
# split independent and dependent variables
x = dataset.iloc[:100, :3].values
y = dataset.iloc[:100, -1].values

In [ ]:
x[:5]

array([[-0.92676711, -0.11107301,  0.08601709],
       [-0.91758292,  0.70600622,  0.05804064],
       [ 0.43798448,  0.89909253,  0.07254295],
       [ 0.08969387,  0.29144614,  0.07044433],
       [ 0.11067169, -0.07080569, -0.09037588]])

In [ ]:
y[:5]

array([0, 0, 0, 1, 1])

#### 3) Training step

In [ ]:
# training process
model = MLPAlgorithm(eta = 0.01, max_epochs = 1000, threshold = 1e-4)
model.build_architecture(input_length = 3, hidden_length = 2, output_length = 1)
model.fit(x, y)

Epoch  100 : loss =  0.12181
Epoch  200 : loss =  0.09523
Epoch  300 : loss =  0.0649
Epoch  400 : loss =  0.05375
Epoch  500 : loss =  0.04964
Epoch  600 : loss =  0.04789
Epoch  700 : loss =  0.04706
Epoch  800 : loss =  0.04663
Epoch  900 : loss =  0.04638
Epoch  1000 : loss =  0.04623


#### 4) Testing step

In [ ]:
# load dataset
dataset_test = pd.read_csv("dataset/moonDataset.csv", header = None)
dataset_test.columns = ['X1','X2','X3','Outcome']
dataset_test

,X1,X2,X3,Outcome
0,-0.926767,-0.111073,0.086017,0
1,-0.917583,0.706006,0.058041,0
2,0.437984,0.899093,0.072543,0
3,0.089694,0.291446,0.070444,1
4,0.110672,-0.070806,-0.090376,1
...,...,...,...,...
195,-0.540630,0.901834,-0.058539,0
196,0.032085,0.411465,-0.064823,1
197,0.691922,0.679103,0.020613,0
198,0.017034,0.930984,-0.034133,0


In [ ]:
x_t = dataset_test.iloc[:, :3].values
y_t = dataset_test.iloc[:, -1].values

In [ ]:
# make test
model.test(x_t, y_t)
print("Accuracy = ", model.accuracy, "%")

Accuracy =  87.0 %
